In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import os
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)

c:\Users\aditya\AppData\Local\Programs\Python\Python310\lib\site-packages\seaborn\_statistics.py:32: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.4)
  from scipy.stats import gaussian_kde


In [3]:
raw_df = pd.read_csv('./Datasets/Data4.csv')
raw_df

,year,month,day,hour,weekend,holiday,temp,precipitation,wind_speed,preasure,electricity_consumed,prev_avg_e
0,2017,1,1,0,1,0,15.85,3.3,5.70,1016.7,1775.69,2396.725833
1,2017,1,1,1,1,0,15.85,0.8,5.70,1016.7,1565.98,2396.725833
2,2017,1,1,2,1,0,15.85,2.8,5.70,1016.7,1458.04,2396.725833
3,2017,1,1,3,1,0,15.85,0.5,5.70,1016.7,1413.88,2396.725833
4,2017,1,1,4,1,0,15.85,0.8,5.70,1016.7,1441.43,2396.725833
...,...,...,...,...,...,...,...,...,...,...,...,...
63979,2024,4,19,19,0,0,31.75,0.0,7.85,1004.1,3413.68,2833.395833
63980,2024,4,19,20,0,0,31.75,0.0,7.85,1004.1,3163.21,2833.395833
63981,2024,4,19,21,0,0,31.75,0.0,7.85,1004.1,2909.03,2833.395833
63982,2024,4,19,22,0,0,31.75,0.0,7.85,1004.1,2686.85,2833.395833


In [4]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(raw_df, test_size=0.2, random_state=42)

inp_cols = ['month', 'hour', 'temp', 'wind_speed', 'preasure', 'prev_avg_e']
target_col = 'electricity_consumed'

train_inputs = train_df[inp_cols].copy()
train_targets = train_df[target_col].copy()

test_inputs = test_df[inp_cols].copy()
test_targets = test_df[target_col].copy()


In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(raw_df[inp_cols])

In [6]:
train_inputs[inp_cols] = scaler.transform(train_inputs[inp_cols])
test_inputs[inp_cols] = scaler.transform(test_inputs[inp_cols])

train_inputs.describe().loc[['min', 'max']]

,month,hour,temp,wind_speed,preasure,prev_avg_e
min,0.0,0.0,0.0,0.0,0.0,0.0
max,1.0,1.0,1.0,1.0,1.0,1.0


In [7]:
from sklearn.ensemble import RandomForestRegressor

In [8]:
model = RandomForestRegressor(n_jobs=-1, random_state=45)

In [9]:
%%time
model.fit(train_inputs, train_targets)

Wall time: 1.82 s


RandomForestRegressor(n_jobs=-1, random_state=45)

In [10]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(test_inputs)

# Evaluate the model
mse = mean_squared_error(test_targets, y_pred)
r2 = r2_score(test_targets, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 25832.52755535915
R^2 Score: 0.9817027550501424


In [11]:
import joblib


joblib.dump(model, "rf_model.pkl")
# joblib.dump(scaler, "rf_scaler.pkl")

['rf_model.pkl']

In [24]:
# testing by loading model from file


example_data = pd.DataFrame({
    'month' : [6],
    'hour' : [14],
    'temp' : [34.25],
    'wind_speed' : [16.1],
    'preasure': [1001.7],
    'prev_avg_e': [5459.494583333334]

})

model = joblib.load("rf_model.pkl")
scaler = joblib.load("rf_scaler.pkl")
example_data[['month', 'hour', 'temp', 'wind_speed', 'preasure', 'prev_avg_e']] = scaler.transform(example_data[['month', 'hour', 'temp', 'wind_speed', 'preasure', 'prev_avg_e']])


model.predict(example_data)

array([6246.7942])